# Advanced MLP - 2
- Putting it altogether

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.wrappers.scikit_learn import KerasClassifier
from keras.datasets import mnist
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Activation, Dense, BatchNormalization, Dropout
from keras import optimizers

Using TensorFlow backend.


## Load Dataset

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 6s 1us/step


In [3]:
# reshaping X data: (n, 28, 28) => (n, 784)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1] * X_test.shape[2]))

In [4]:
# We use all training data and validate on all test data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 784)
(10000, 784)
(60000,)
(10000,)


## Training & Validating Model
- Measures to improve training is applied simultaneously
    - More training set
    - Weight Initialization scheme
    - Nonlinearity (Activation function)
    - Optimizers: adaptvie
    - Batch Normalization
    - Dropout (Regularization)
    - Model Ensemble

In [5]:
def mlp_model():
    model = Sequential()
    
    model.add(Dense(50, input_shape = (784, ), kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))    
    model.add(Dropout(0.2))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, kernel_initializer='he_normal'))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [6]:
# create 5 models to ensemble
model1 = KerasClassifier(build_fn = mlp_model, epochs = 100)
model2 = KerasClassifier(build_fn = mlp_model, epochs = 100)
model3 = KerasClassifier(build_fn = mlp_model, epochs = 100)
model4 = KerasClassifier(build_fn = mlp_model, epochs = 100)
model5 = KerasClassifier(build_fn = mlp_model, epochs = 100)

In [7]:
ensemble_clf = VotingClassifier(estimators = [('model1', model1), ('model2', model2), ('model3', model3), ('model4', model4), ('model5', model5)], voting = 'soft')

In [8]:
ensemble_clf.fit(X_train, y_train)

Epoch 1/100
60000/60000 [==============================] - 23s 385us/step - loss: 0.7667 - accuracy: 0.7646
Epoch 2/100
60000/60000 [==============================] - 17s 286us/step - loss: 0.4399 - accuracy: 0.8742
Epoch 3/100
60000/60000 [==============================] - 18s 300us/step - loss: 0.3669 - accuracy: 0.8955
Epoch 4/100
60000/60000 [==============================] - 17s 279us/step - loss: 0.3264 - accuracy: 0.9058
Epoch 5/100
60000/60000 [==============================] - 17s 289us/step - loss: 0.3019 - accuracy: 0.9148
Epoch 6/100
60000/60000 [==============================] - 17s 280us/step - loss: 0.2818 - accuracy: 0.9202
Epoch 7/100
60000/60000 [==============================] - 17s 288us/step - loss: 0.2670 - accuracy: 0.9248
Epoch 8/100
60000/60000 [==============================] - 16s 270us/step - loss: 0.2562 - accuracy: 0.9281s - loss: 0.2572 
Epoch 9/100
60000/60000 [==============================] - 16s 268us/step - loss: 0.2503 - accuracy: 0.9291
Epoch 10/10

KeyboardInterrupt: 

In [9]:
y_pred = ensemble_clf.predict(X_test)

 9984/10000 [============================>.] - ETA: 0s

In [10]:
print('Acc: ', accuracy_score(y_pred, y_test))

Acc:  0.9794


Final model produces near **98% of accuracy**, which is rather improved from before

In [11]:
y_train.shape

(60000,)

In [12]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)